# Prerequisites

In [318]:
!pip install -U selenium
!pip install python-docx
!pip install pyldavis

In [ ]:
# Getting some weird deprecation warnings? Hopefully will suppress
import warnings
warnings.filterwarnings('ignore')

# Using Selenium to scrape OPs
## Requires a browser and corresponding driver
Navigate to 
https://selenium-python.readthedocs.io/installation.html#drivers

A driver allows a web browser that you have installed to be automatically ran.
Download one of the drivers. The driver may come in a .zip file which you may need to unzip.
Place the downloaded driver inside the same directory of this notebook and change the constants below.

In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By


# Change USING_CHROME to False if you're not using Chrome
USING_CHROME = True

# Change driver path with the path to your driver
DRIVER_PATH = 'chromedriver.exe'

# Change "Chrome" to another browser i.e. "Firefox" if not using Chrome
driver = webdriver.Chrome(DRIVER_PATH)
if (USING_CHROME):
    # Allow multiple downloads - chrome only
    chrome_options = webdriver.ChromeOptions()

    prefs = {'profile.default_content_setting_values.automatic_downloads': 1}
    chrome_options.add_experimental_option("prefs", prefs)
    driver = webdriver.Chrome(options = chrome_options) 
    
driver.get('https://dfafacts.gov/');

# Click the "I understand" button
elem = driver.find_element(By.CSS_SELECTOR, 'input')
elem.click()

# Select the login name
#usern = driver.find_element(By.CSS_SELECTOR, 'input#loginName')
#passw = driver.find_element(By.CSS_SELECTOR, 'input#Password')
#authc = driver.find_element(By.CSS_SELECTOR, 'input[id="Authenticator Code"]')
#logb = driver.find_element(By.CSS_SELECTOR, 'button#btnLogin')


<ipython-input-1-55dacbc708e1>:13: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(DRIVER_PATH)


## Login

Login directly by opening up the browser and entering credentials

## Downloading OPs
You may need to enable "Download Multiple Files" when prompted if your browser requests it.
The downloaded files will be in your default download location i.e. Downloads

In [215]:
import time

# Gotta add a delay cause FactsInfo is so laggy
# To reduce errors, increase the value of DELAY
DELAY = 1

# Helper function to repeatedly try to click
# Tries 10 times before stopping
def click(CSSorElem, lag=1):
    attempt = 0
    while attempt < 10:
        try:
            if type(CSSorElem) == str:
                driver.find_element(By.CSS_SELECTOR, CSSorElem).click()
            else:
                CSSorElem.click()
            attempt = 99
        except Exception as e:
            print("Could not click... " + str(e))
            time.sleep(DELAY * lag)
        attempt += 1
    
    # How much to delay after finishing
    time.sleep(DELAY * lag)
    return attempt


# Click the OP button
click('i.fa-rocket', lag=3)

# Click reports
click('div.col-md-12 > ul > li:nth-child(5) > a')

# Get all options for "Bureau"
bureaus = driver.find_elements(By.CSS_SELECTOR, 'select#selOpBureau > option')
for bureau in bureaus:
    click(bureau, lag=2)
    
    # Get all options for "Operational Unit"
    units = driver.find_elements(By.CSS_SELECTOR, 'select#selOpUnit > option')
    for unit in units:
        click(unit)
        print(bureau.text + " - " + unit.text)
        
        # Attempt to download the operational plan 
        click('i[title="Click to Generate Full Operation Plan Report"]', lag=2)
print("Done")

Africa - African Union
Africa - Angola
Africa - Benin
Africa - Botswana
Africa - Burkina Faso
Africa - Burundi
Africa - Cabo Verde
Africa - Cameroon
Could not click... Message: element click intercepted: Element <i smart-button-click="vm.openFullReport(false)" title="Click to Generate Full Operation Plan Report" alt="Click to Generate Full Operation Plan Report" class="fa fa-file-word-o fa-2x btn-link"></i> is not clickable at point (477, 302). Other element would receive the click: <div class="page-splash-message page-splash-message-subtle ng-binding">...</div>
  (Session info: chrome=111.0.5563.65)
Stacktrace:
Backtrace:
	(No symbol) [0x00E4DCE3]
	(No symbol) [0x00DE39D1]
	(No symbol) [0x00CF4DA8]
	(No symbol) [0x00D252EA]
	(No symbol) [0x00D23BBA]
	(No symbol) [0x00D21EFB]
	(No symbol) [0x00D21027]
	(No symbol) [0x00D18D05]
	(No symbol) [0x00D3AECC]
	(No symbol) [0x00D18726]
	(No symbol) [0x00D3B224]
	(No symbol) [0x00D4D57C]
	(No symbol) [0x00D3ACC6]
	(No symbol) [0x00D16F68]
	(No 

KeyboardInterrupt: 

# OP Text Processing
Organize each document into text paragraphs, tables, and implementing mechanisms. 

**You must manually move your downloaded OPs into a folder titled "OP" in the directory that you are running this jupyter notebook**

In [2]:
import docx
import glob
import re
import pandas as pd
import numpy as np


# Get all documents, turn each document into [paragraph, table] objects
documents = []
for file_name in glob.glob('./OP/*.docx'):
    doc = docx.Document(file_name)              
    documents.append([doc.paragraphs, doc.tables])    

    
# Turn it into a more readable object
# Each document has a "title" with attributes "par" pargraphs, "table" tables, and "im", implementing mechanisms
# pdocs is a dictionary: {"title" : {"par": paragraphs, "tab" : tables, "im" : list of ims}}
# ims: [table, string]   
pdocs = {}
for i in range(len(documents)):
    title = ""
    for j in range(len(documents[i][0])):
        if re.search("[A-z]+", documents[i][0][j].text):
            title = documents[i][0][j].text 
            break
    pdocs[title] = {"par" : documents[i][0], "tab" : documents[i][1], "im" : []}
print("Done")

Done


In [3]:
# Build the list of implementing mechanism tables
for title in pdocs:
    # Wipe 
    pdocs[title]["im"] = []
    p_last = 0
    for i in range(len(pdocs[title]["tab"])):
        table = pdocs[title]["tab"][i]
        #print("%s %s" % (len(table.rows), len(table.columns)))
        
        # Catch bugged tables
        try:
            table.cell(0, 0)
        except:
            print(title, "phantom table")
            continue
        
        
        if table.cell(0, 0).text.find("Mechanism") != -1:
            # Found an IM table, now find the corresponding paragraph with IM table
            # Get im number
            im_num = table.cell(0, 1).text
            
            # Find the corresponding summary.
            pars = pdocs[title]["par"]
            for j in range(p_last, len(pars)):
                if pars[j].text.find(im_num) != -1:
                    break
                    
            if j == len(pars) - 1:
                print("ERROR: %s matching paragraph could not be found" % im_num)
                pdocs[title]["im"].append([table, "ERROR"])
                continue
            
            # Append all text corresponding to the table
            corr_par = ""
            for j2 in range(j+1, len(pars)):
                # Skip this line
                if pars[j2].text.find("IMPLEMENTING MECHANISM NARRATIVE") != -1 or re.match("[\n ]+", pars[j].text):
                    continue
                
                # Stop at "FUNDING SUMMARY"
                if pars[j2].text.find("FUNDING SUMMARY") != -1:
                    break
                
                corr_par += pars[j2].text.strip() + "\n"
            
            # Try to optimize it a bit
            p_last = j
            
            # Append the table, summary
            pdocs[title]["im"].append([table, corr_par])         
print("Done")

Togo phantom table
Done


In [67]:
# User Helper functions

# Print out a IM table
def print_im(table):
    # IM will always have 2 columns
    for i in range(len(table.rows)):
        print("%-30s %s" % (table.cell(i, 0).text, table.cell(i, 1).text))
    print()

# Return a flat list of contents of an implementing mechanism table
# This returns a standard list of 12 columns. Some tables do not have 12 columns, specifically lacking "PPARM"
# then the value of None is inserted
def flat_im(table):
    fl = [x.text for x in table.column_cells(1)]
    if len(fl) == 11:
        # Likely missing PPARM
        fl.insert(7, None)
    elif len(fl) < 11 or len(fl) > 12:
        print("Warning: unrecognized size of table")
    return fl
    

# Print out all implementing mechanisms for a document
# doc: document name as a string
# num: the first num ims to print
def print_im_doc(doc, num):
    i = 0
    for ims in pdocs[doc]["im"]:
        if i == num:
            break
        print_im(ims[0])
        print(ims[1])
        i += 1

In [185]:
# Example: print out first 5 IM tables for Afhganistan
print_im_doc("Afghanistan", 5)

Mechanism Number               46246
Implementing Mechanism Name:   Central Contraceptives Procurement (CCP), field support mechanism
Prime Partner:                 John Snow, Incorporated 
Award Number:                  306-AID-OAA-TO-10-00066
Implementing Mechanism Type:   Direct Contracts
Source Agency:                 U.S. Agency for International Development
Implementing Agency:           U.S. Agency for International Development
Planned Funding:               $3,000,000
Start Date:                    04/20/2015
End Date:                      11/28/2023
Total Estimated Cost:          $25,000,000

The Global Health Supply Chain- Procurement and Supply Management Project (GHS-PSM) serves as the central procurement mechanism for USAID Missions worldwide to purchase high quality contraceptives and other essential public health supplies. Through various contractors, this activity implements the USAID policy of centralized contraceptive procurement by providing a simplified mechanism fo

In [68]:
# Convert IM table data into a CSV
import numpy as np

# Get a giant list of all im tables flattened with an additional feature of the document name it came from
master = []
for doc in pdocs:
    for im in pdocs[doc]["im"]:
        master += [[doc] + flat_im(im[0]) + [im[1]]]

        
# Should I include the document? Should I shorthand the name?
im_df = pd.DataFrame(master, columns=["Document", "Mechanism Number", "Name", "Partner", 
                                      "Award Number", "Type", "Source Agency", "Implementing Agency",
                                      "PPARM", "Planned Funding", "Start Date", "End Date", "Total Estimated Cost",
                                      "Narrative"])

In [70]:
# Convert to CSV
im_df.to_csv('OPIMs.csv') 

In [106]:
im_df

,Document,Mechanism Number,Name,Partner,Award Number,Type,Source Agency,Implementing Agency,PPARM,Planned Funding,Start Date,End Date,Total Estimated Cost,Narrative
0,Afghanistan,46246,"Central Contraceptives Procurement (CCP), fiel...","John Snow, Incorporated",306-AID-OAA-TO-10-00066,Direct Contracts,U.S. Agency for International Development,U.S. Agency for International Development,None,"$3,000,000",04/20/2015,11/28/2023,"$25,000,000",The Global Health Supply Chain- Procurement an...
1,Afghanistan,46786,PDS - Small Grants Program,To be Determined - Unknown,To be Determined,Direct Grant/Cooperative Agreement,U.S. Department of State,U.S. Department of State,None,"$3,000,000",,,"$7,000,000",The Public Diplomacy Section’s (PDS) Small Gra...
2,Afghanistan,46809,INL/Program Support,To be Determined - Unknown,IN21AF12,Other USG Direct,U.S. Department of State,U.S. Department of State,None,"$6,000,000",10/01/2021,09/30/2026,"$12,000,000",These funds will support program development a...
3,Afghanistan,100772,Counter-Trafficking In Persons (CTIP)-III,INTERNATIONAL ORGANIZATION FOR MIGRATION,To be Determined,Direct Grant/Cooperative Agreement,U.S. Agency for International Development,U.S. Agency for International Development,None,"$2,137,347",02/01/2023,01/30/2025,"$6,500,000","With FY 2022 funds, USAID will continue to fun..."
4,Afghanistan,102903,USAID/Afghanistan Program Development and Admi...,To be Determined - Unknown,To be Determined,Direct Contracts,U.S. Agency for International Development,U.S. Agency for International Development,None,"$7,220,712",09/28/2022,09/27/2023,"$21,230,500",USAID will utilize FY 2022 funds for a variety...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4238,Zimbabwe,113484,NEW - Agriculture/Environment Institutional Ca...,To be Determined - Unknown,To be Determined,To be Determined,U.S. Agency for International Development,U.S. Agency for International Development,None,"$1,000,000",10/02/2023,09/29/2028,"$5,000,000",This new activity will build the institutional...
4239,Zimbabwe,113497,NEW - PMI VectorLink II Malaria Surveillance A...,To be Determined - Unknown,To be Determined,To be Determined,U.S. Agency for International Development,U.S. Agency for International Development,None,"$100,000",,,-,"The VectorLink follow-on project, a mechanism ..."
4240,Zimbabwe,113498,NEW - Health Supply Chain Central Mechanism,To be Determined - Unknown,To be Determined,To be Determined,U.S. Agency for International Development,U.S. Agency for International Development,None,"$300,000",03/01/2023,02/28/2026,"$1,000,000",This Washington-managed mechanism is within th...
4241,Zimbabwe,113499,"NEW - Maternal, Newborn, and Child Health",To be Determined - Unknown,To be Determined,To be Determined,U.S. Agency for International Development,U.S. Agency for International Development,None,"$3,258,800",04/30/2023,09/30/2027,"$18,000,000","The new maternal, newborn, and child health (M..."


# Keyword Analysis
The keywords file must be placed in the directory that you are running this notebook.

In [79]:
import re

# The format of the keywords file should be
# name
# keyword|keyword2|keyword3              (on a single line)
kwords = {}
with open('keywords.txt') as k:
    kdoc = k.read()
    spl = re.compile("\n").split(kdoc)
    
    # A simple algorithm that matches the previous non-empty line
    tag_name = None
    for s in spl:
        if not re.match("\s+", s):
            if tag_name:
                kwords[tag_name] = s
                tag_name = None
            else:
                tag_name = s
kwords.keys()

dict_keys(['Civil Society', 'Governance', 'Corruption', 'Human Rights', 'Independent Media and Free Flow of Information', 'Political Competition and Consensus Building', 'Rule of Law', "Women's Political Participation and Leadership", 'Youth', 'Environment & Climate Change', 'Migration', 'Misinformation'])

In [94]:
# Add keywords to the existing implementing mechanisms dataframe
kw_df = im_df.loc[:, ["Document", "Mechanism Number", "Narrative"]]
for k in kwords.keys():
    kw_df[k] = kw_df.apply(lambda row: len(re.findall(kwords[k], row["Narrative"])), axis=1)
kw_df = kw_df.drop(["Narrative"], axis=1)

In [95]:
kw_df

,Document,Mechanism Number,Civil Society,Governance,Corruption,Human Rights,Independent Media and Free Flow of Information,Political Competition and Consensus Building,Rule of Law,Women's Political Participation and Leadership,Youth,Environment & Climate Change,Migration,Misinformation
0,Afghanistan,46246,0,4,0,0,0,0,0,0,0,0,0,0
1,Afghanistan,46786,4,1,0,1,5,0,0,3,1,0,0,0
2,Afghanistan,46809,0,1,0,0,0,0,0,0,0,0,0,0
3,Afghanistan,100772,2,1,0,3,1,0,1,0,0,0,0,0
4,Afghanistan,102903,0,1,0,0,1,0,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4238,Zimbabwe,113484,4,3,0,3,0,0,0,3,0,5,0,0
4239,Zimbabwe,113497,1,1,0,1,0,0,0,0,0,1,0,0
4240,Zimbabwe,113498,0,4,0,1,0,0,1,0,0,0,0,0
4241,Zimbabwe,113499,0,0,0,1,0,0,0,0,1,0,0,0


In [ ]:
kw_df.to_csv("keywords.csv")

# Topic Modelling
Transform implementing mechanism descriptions into bag of words and use LDA to construct topics

In [188]:
# Get the text contents from implementing mechanism table
im_docs = []

# Get a title for each implementing mechanism table
titles = []
for title in pdocs:
    ims = pdocs[title]["im"]
    for im in ims:
        # Currently first three letters + table ID
        titles.append(title[:3] + " " + im[0].cell(0, 1).text)
        im_docs.append(im[1])

In [191]:
import pandas as pd
import sklearn.feature_extraction as skft
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.decomposition import LatentDirichletAllocation
import pyLDAvis
import pyLDAvis.lda_model
import numpy as np

ADD_STOPWORDS = ["usaid", "usaids", "agency", 
                 "international", "development", 
                 "unclassified", "embassy", "approved", 
                 "public", "release", "cdcs", "mission", "country"]
stwords = list(skft.text.ENGLISH_STOP_WORDS.union(ADD_STOPWORDS))

# Convert documents into bag of words
# throw out words that do noy appear in 3% of docs
vectorizer = CountVectorizer(stop_words=stwords, lowercase=True, min_df=0.03)
raw_bag = vectorizer.fit_transform(im_docs)
bag = pd.DataFrame(data=raw_bag.toarray(), columns = vectorizer.get_feature_names_out(), index=titles)

tfidf_vect = TfidfTransformer()
tfidf_bag = tfidf_vect.fit_transform(bag)
tfidf = pd.DataFrame(data=tfidf_bag.toarray(), columns = vectorizer.get_feature_names_out(), index=titles)

In [190]:
# Bag of words sample output
bag

,000,19,2020,2021,2022,2023,500,ability,able,access,...,work,workers,workforce,working,works,world,year,years,young,youth
Afg 46246,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Afg 46786,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,1
Afg 46809,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Afg 100772,0,0,0,0,1,0,0,0,0,3,...,0,0,0,0,0,0,0,0,0,0
Afg 102903,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Zim 113484,0,0,0,0,0,0,0,0,0,3,...,0,0,0,0,0,0,0,0,0,0
Zim 113497,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
Zim 113498,0,0,0,0,1,0,0,0,0,1,...,1,0,0,0,0,0,0,0,0,0
Zim 113499,0,0,0,0,1,0,0,0,0,1,...,0,1,0,0,0,0,0,0,0,0


In [205]:
# Create a model
bag_lda = LatentDirichletAllocation(n_components=7)
bag_lda.fit(raw_bag)

LatentDirichletAllocation(n_components=7)

In [206]:
# Visualize results
pyLDAvis.lda_model.prepare(bag_lda, raw_bag, vectorizer)

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
4      0.013417  0.085679       1        1  21.443500
0      0.004024 -0.165823       2        1  17.892773
2      0.121575 -0.038839       3        1  14.534323
3     -0.140137 -0.054723       4        1  13.360066
5     -0.104376  0.091491       5        1  13.339426
1      0.010468  0.020016       6        1  12.354062
6      0.095029  0.062200       7        1   7.075849, topic_info=           Term         Freq        Total Category  logprob  loglift
248      health  5434.000000  5434.000000  Default  30.0000  30.0000
526          tb  1326.000000  1326.000000  Default  29.0000  29.0000
551       water   910.000000   910.000000  Default  28.0000  28.0000
164   education  1596.000000  1596.000000  Default  27.0000  27.0000
16     activity  6078.000000  6078.000000  Default  26.0000  26.0000
..          ...          ...          ...      ...      ...      ...
230          fy   148.245532  2094.535894   Topic7  -4.9354   0.0003
78     capacity   162.366210  2894.136701   Topic7  -4.8444  -0.2321
429     provide   132.826781  1872.824121   Topic7  -5.0452   0.0023
241  government   132.413127  2111.850761   Topic7  -5.0484  -0.1209
228       funds   130.671900  2491.431790   Topic7  -5.0616  -0.2994

[391 rows x 6 columns], token_table=      Topic      Freq   Term
term                        
0         1  0.064942    000
0         2  0.349281    000
0         3  0.301891    000
0         4  0.135149    000
0         5  0.018429    000
...     ...       ...    ...
559       7  0.108387  world
563       1  0.012016  youth
563       2  0.578363  youth
563       3  0.186646  youth
563       6  0.222694  youth

[1497 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[5, 1, 3, 4, 6, 2, 7])

In [209]:
# Get predicted results for each document
bag_topic_dist = bag_lda.transform(raw_bag)

# Get the top topic prediction for each document
top_topic = bag_topic_dist.argsort(1)[:, [-1]]

# Quick and dirty unvectorized way of getting corresponding probabilities
# Not sure how to do it the numpy way
top_prob = []
for i in range(len(bag_topic_dist)):
    top_prob.append(bag_topic_dist[i,top_topic[i]][0])

# Turn into dataframe
topic_df = pd.DataFrame(data={"topic":top_topic.flatten(), "probability":top_prob}, index=titles)
topic_df

,topic,probability
Afg 46246,4,0.892931
Afg 46786,0,0.685278
Afg 46809,1,0.954766
Afg 100772,0,0.513422
Afg 102903,3,0.730577
...,...,...
Zim 113484,2,0.509865
Zim 113497,4,0.721882
Zim 113498,4,0.786649
Zim 113499,4,0.986777


In [210]:
# User helper function
# Get all topics associated with each document
def get_tg_topics(df, regex):
    r = re.compile(regex)
    #return topic_df.loc[list(filter(r.match, topic_df.index)), "topic"]
    return topic_df.loc[list(filter(r.match, topic_df.index)), "topic"].value_counts()

# Return all three letter base tags from the 
def get_document_tags(df)
    return set([x[:3] for x in df.index])

In [211]:
get_tg_topics(topic_df, "Afg.*")

4    6
0    6
1    6
2    4
3    3
6    2
Name: topic, dtype: int64

In [213]:
# Right now the topics from the visualization and the topics from the transformed model don't match up
# Use below to find out which corresponds to which
for topic in range(0, len(lda2.components_)):
    print(topic)
    print("".join(bag.columns[x] + " " for x in lda2.components_[topic].argsort()[-10:]))
    #a = lda2.components_[topic].copy()
    #a.sort()
    #print(a[-10:])
    #print()

0
trafficking justice enforcement military capacity support law training security tb 
1
business increase growth youth opportunities women sector private economic activity 
2
provide funding staff programs activities assistance technical funds program support 
3
regional resilience activities security global support research countries climate food 
4
support resources access sustainable local improve management services water activity 
5
democratic local activity program media support human civil society rights 
6
threats capacity risk diseases national global support security health energy 
7
systems 2022 national services improve quality malaria support activity health 
8
quality school improve violence children support youth learning activity education 
